In [1]:
from pyspark.sql import SparkSession
import pandas as pd

In [3]:
# ---- 1️⃣ Install Java 17 + Spark 3.5.1 ----
!apt-get install openjdk-17-jdk-headless -qq > /dev/null
!wget -q -O spark.tgz https://archive.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar -xzf spark.tgz -C /content/
!pip install -q pyspark==3.5.1 findspark

# ---- 2️⃣ Set environment variables ----
import os, findspark
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"
os.environ["HADOOP_USER_NAME"] = "root"
findspark.init("/content/spark-3.5.1-bin-hadoop3")

# ---- 3️⃣ Create Spark session ----
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Demo02") \
    .master("local[*]") \
    .getOrCreate()

print("✅ Spark session started successfully!")
spark

✅ Spark session started successfully!


In [4]:
df = spark.read.csv('tips.csv', header=True, inferSchema=True)

In [5]:
df.show(10)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
+----------+----+------+------+---+------+----+
only showing top 10 rows



In [6]:
df.printSchema()

root
 |-- total_bill: double (nullable = true)
 |-- tip: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- day: string (nullable = true)
 |-- time: string (nullable = true)
 |-- size: integer (nullable = true)



In [7]:
# Handling Categorical Features
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="sex",outputCol="sex_indexed")
df_r = indexer.fit(df).transform(df)
df_r.show()

+----------+----+------+------+---+------+----+-----------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|
+----------+----+------+------+---+------+----+-----------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|        0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|        0.0|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|        0.0|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|        0.0|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|        0.0|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|        0.0|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|        1.0|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|        0.0|
|     18.43| 3.0|  Male|    No|Sun|Dinne

In [8]:
# Handling Categorical Features
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCols=["smoker","day","time"],outputCols=["smoker_indexed","day_indexed","time_indexed"])
df_r = indexer.fit(df).transform(df)
df_r.show()

+----------+----+------+------+---+------+----+--------------+-----------+------------+
|total_bill| tip|   sex|smoker|day|  time|size|smoker_indexed|day_indexed|time_indexed|
+----------+----+------+------+---+------+----+--------------+-----------+------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|           0.0|        1.0|         0.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|           0.0|        1.0|         0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|           0.0|        1.0|         0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|           0.0|        1.0|         0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|           0.0|        1.0|         0.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|           0.0|        1.0|         0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|           0.0|        1.0|         0.0|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|           0.0|        1.0|         0.0|
|     15.04|1.96|  Male|    No|S

In [12]:
from pyspark.ml.feature import StringIndexer, VectorAssembler

# 1️⃣ Create indexers
indexer_sex = StringIndexer(inputCol="sex", outputCol="sex_indexed")
indexer_smoker = StringIndexer(inputCol="smoker", outputCol="smoker_indexed")
indexer_day = StringIndexer(inputCol="day", outputCol="day_indexed")
indexer_time = StringIndexer(inputCol="time", outputCol="time_indexed")

# 2️⃣ Apply them sequentially
for col_name in ["sex_indexed", "smoker_indexed", "day_indexed", "time_indexed"]:
    if col_name in df_r.columns:
        df_r = df_r.drop(col_name)
df_r = indexer_sex.fit(df_r).transform(df_r)
df_r = indexer_smoker.fit(df_r).transform(df_r)
df_r = indexer_day.fit(df_r).transform(df_r)
df_r = indexer_time.fit(df_r).transform(df_r)

# 3️⃣ Now create feature vector
featureassembler = VectorAssembler(
    inputCols=['tip','size','sex_indexed','smoker_indexed','day_indexed','time_indexed'],
    outputCol="Independent Features"
)
output = featureassembler.transform(df_r)

output.show(5)

+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+--------------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|smoker_indexed|day_indexed|time_indexed|Independent Features|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+--------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|           0.0|        1.0|         0.0|[1.01,2.0,1.0,0.0...|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|[1.66,3.0,0.0,0.0...|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|[3.5,3.0,0.0,0.0,...|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|[3.31,2.0,0.0,0.0...|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|           0.0|        1.0|         0.0|[3.61,4.0,1.0,0.0...|
+----------+----+------+------+-

In [13]:
output.select('Independent Features').show()

+--------------------+
|Independent Features|
+--------------------+
|[1.01,2.0,1.0,0.0...|
|[1.66,3.0,0.0,0.0...|
|[3.5,3.0,0.0,0.0,...|
|[3.31,2.0,0.0,0.0...|
|[3.61,4.0,1.0,0.0...|
|[4.71,4.0,0.0,0.0...|
|[2.0,2.0,0.0,0.0,...|
|[3.12,4.0,0.0,0.0...|
|[1.96,2.0,0.0,0.0...|
|[3.23,2.0,0.0,0.0...|
|[1.71,2.0,0.0,0.0...|
|[5.0,4.0,1.0,0.0,...|
|[1.57,2.0,0.0,0.0...|
|[3.0,4.0,0.0,0.0,...|
|[3.02,2.0,1.0,0.0...|
|[3.92,2.0,0.0,0.0...|
|[1.67,3.0,1.0,0.0...|
|[3.71,3.0,0.0,0.0...|
|[3.5,3.0,1.0,0.0,...|
|(6,[0,1],[3.35,3.0])|
+--------------------+
only showing top 20 rows



In [15]:
finalized_data = output.select('Independent Features','total_bill')

In [16]:
from pyspark.ml.regression import LinearRegression
train_data, test_data = finalized_data.randomSplit([0.75, 0.25])
regressor = LinearRegression(featuresCol='Independent Features', labelCol='total_bill')
regressor = regressor.fit(train_data)

In [17]:
regressor.coefficients

DenseVector([3.1845, 3.5156, -1.8082, 2.6213, -0.4007, -0.1386])

In [18]:
regressor.intercept

1.698605583444052

In [19]:
pred_results = regressor.evaluate(test_data)

In [20]:
pred_results.predictions.show()

+--------------------+----------+------------------+
|Independent Features|total_bill|        prediction|
+--------------------+----------+------------------+
|(6,[0,1],[1.45,2.0])|      9.55|13.347311720205296|
|(6,[0,1],[1.47,2.0])|     10.77| 13.41100199091984|
|(6,[0,1],[1.97,2.0])|     12.02|15.003258758783401|
|(6,[0,1],[2.24,3.0])|     16.04| 19.37865816840818|
| (6,[0,1],[2.5,4.0])|     18.35|23.722212442675687|
|(6,[0,1],[2.64,3.0])|     17.59| 20.65246358269903|
|(6,[0,1],[2.72,2.0])|     13.28|17.391643910578747|
|(6,[0,1],[3.15,3.0])|     20.08|22.276565485919864|
| (6,[0,1],[4.3,2.0])|      21.7| 22.42317529702761|
| (6,[0,1],[5.0,3.0])|     31.27| 28.16791552701505|
|(6,[0,1],[5.92,3.0])|     29.03|31.097667979884005|
|[1.0,1.0,1.0,0.0,...|      7.25| 6.590514277125094|
|[1.44,2.0,0.0,0.0...|      7.56|12.375430752140717|
|[1.44,2.0,0.0,1.0...|      7.74|15.936772908976113|
|[1.48,2.0,0.0,0.0...|      8.52|  12.5028112935698|
|[1.5,2.0,0.0,1.0,...|     11.59| 16.127843721

In [22]:
pred_results.r2

0.5139674994301371

In [23]:
pred_results.meanSquaredError, pred_results.meanAbsoluteError

(33.9229934513366, 4.354990289924728)